## Code to train the BERT-NLI Outgroup Hostility model
### **❗This notebook was copied from the Demo Notebook for Less Annotating, More Classifying. Most helpful comments in the code came from the Demo Notebook. The credit for the code and most comments belongs to the authors of this paper: https://doi.org/10.1017/pan.2023.20.❗**

Read more about the BERT-NLI approach used here: ["Less Annotating, More Classifying: Addressing the Data Scarcity Issue of Supervised Machine Learning with Deep Transfer Learning and BERT-NLI"](https://github.com/MoritzLaurer/less-annotating-with-bert-nli) by Moritz Laurer, Wouter van Atteveldt, Andreu Casas, Kasper Welbers.


## Activate a GPU runtime

In order to run this notebook on a GPU, click on "Runtime" > "Change runtime type" > select "GPU" in the menue bar in to top left. Training a Transformer is much faster on a GPU. Given Google's usage limits for GPUs, it is advisable to first test your non-training code on a CPU (Hardware accelerator "None" instead of GPU) and only use the GPU once you know that everything is working.

## Install relevant packages

In [ ]:
!pip install transformers[sentencepiece]==4.23
!pip install datasets==2.6
!pip install optuna==3.0

In [ ]:
## Load general packages
# some more specialised packages are loaded in each sub section
import pandas as pd
import numpy as np
from google.colab.data_table import DataTable

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set random seed for reproducibility
SEED_GLOBAL = 42
np.random.seed(SEED_GLOBAL)

## Download and prepare data

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/ukraine/data-social-id/CSV FILE HERE")

In [ ]:
df_train.columns

Index(['Unnamed: 0', 'X', 'date', 'user_id', 'Zperiod', 'lang', 'source',
       'text', 'descUAid', 'descRUid', 'solUAid', 'hostRUid', 'hostUAid',
       'solRUid', 'Unnamed: 14'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
sample_size = 2000
df_train, df_test = train_test_split(df_train[:sample_size], test_size=.2, random_state=SEED_GLOBAL,stratify=df_train['hostRUid'][:sample_size])
#note: because here we strarified the data by the training label the solidarity and hostility test sets are not the same
print("Length of training and test sets after sampling: ", len(df_train), " (train) ", len(df_test), " (test).")

Length of training and test sets after sampling:  1600  (train)  400  (test).


In [ ]:
df_train['label_text'] = df_train.hostRUid.apply(lambda x: 'Hostility towards Russia or Russians' if x else "Other")
df_test['label_text'] = df_test.hostRUid.apply(lambda x: 'Hostility towards Russia or Russians' if x else "Other")
df_train['label'] = df_train.hostRUid.apply(lambda x: 1 if x else 0)
df_test['label'] = df_test.hostRUid.apply(lambda x: 1 if x else 0)

In [ ]:
#df_test.to_csv("/content/drive/MyDrive/ Exporting Hostility Test Set")

In [ ]:
## inspect the data
# label distribution train set
print("Train set label distribution: ", df_train.label_text.value_counts())
# label distribution test set
print("Test set label distribution: ", df_test.label_text.value_counts())

# full training data table
DataTable(df_train, num_rows_per_page=5)

#Train set label distribution:  Other                                   1290
#Hostility towards Russia or Russians     310
#Name: label_text, dtype: int64
#Test set label distribution:  Other                                   322
#Hostility towards Russia or Russians     78
#Name: label_text, dtype: int64

**If you want to run the notebook on your own dataset:**

You can load your own training and test data above to fine-tune your own BERT-NLI model. Your own dataframe only needs two columns to be compatible with the code below: (1) a "label_text" column with the label texts of your classes, (2) a "text_prepared" column with the texts for training (you might need to delete/adapt the text preparation code cell below for your dataset).

## Create NLI hypotheses

**Formulate a hypothesis, which verbalises the classes/task you are interested in.**

For this example, we base our task on the Manifesto Project codebook:  https://manifesto-project.wzb.eu/coding_schemes/mp_v4

We store the hypothesis in a dictionary: The keys of the dictionary should be the names of the respective label from the training dataframe ('label_text' column); the values of the dictionary should be your manually formulated hypothesis linked to the respective labels.

In [ ]:
# dictionary mapping the dataset's label to manually formulated hypotheses based on the codebook
hypothesis_label_dic = {
    "Other": "The quote is not about hostility towards Russia or Russians",
    "Hostility towards Russia or Russians": "The quote is expressing hostility, derogation or disliking of Russia or Russians, for example it criticises Russia, mentions Russians as incompetent, immoral people or the Russian Federation as a bad, weak or failing nation.",
}

**Prepare the input text**

1.) We prepare the target texts by making them more naturally fit to the hypothesis. Here we simply wrap each target text into the string ' The quote: "{target_text}" - end of the quote. '

2.) We surround the target text by its preceeding and following sentence. Adding context like this systematically increases performance.


In [ ]:
import re
df_train["text_prepared"] = 'The quote: "' + df_train.text.apply(lambda x: re.sub(r"http\S+", "<url>", str(x)), 1) + '" - end of the quote.'
df_test["text_prepared"] =  'The quote: "' +  df_test.text.apply(lambda x: re.sub(r"http\S+", "<url>", str(x)), 1) + '" - end of the quote.'


## Format the training and test datasets for NLI classification


**Format the training data**

1.) For each text with a specific class (label), the corresponding class-hypothesis needs to be added in the same row with the label 'true' (also expressed with the numeric label value 0).

2.) Adding 'false' examples: The NLI task consists of predicting, whether a hypothesis is true or false given a context.
If we only give 'true' hypothesis-context pairs to the algorithm, it will not learn the 'false' class properly.
For each text, we therefore also add a row where the text is matched with a random wrong class label and give it the NLI label 'false' (also expressed with the numeric label value 1). This increases the training data by up to 2x.

See the table below for the concrete format the training data takes after this pre-processing step.
Note that NLI can be formulated as a 3-class (entailment/neutral/contradiction) or 2-class (entailment/not-entailment) task. Both can be used here. We use the 2-class variant.
Note that the words entailment/neutral/contradition and true/neutral/false are used interchangably here. Both terminologies are used in the literature and coding instructions.


In [ ]:
## function for reformatting the train set
def format_nli_trainset(df_train=None, hypo_label_dic=None, random_seed=42):
  print(f"Length of df_train before formatting step: {len(df_train)}.")
  length_original_data_train = len(df_train)

  df_train_lst = []
  for label_text, hypothesis in hypo_label_dic.items():
    ## entailment
    df_train_step = df_train[df_train.label_text == label_text].copy(deep=True)
    df_train_step["hypothesis"] = [hypothesis] * len(df_train_step)
    df_train_step["label"] = [0] * len(df_train_step)
    ## not_entailment
    df_train_step_not_entail = df_train[df_train.label_text != label_text].copy(deep=True)
    df_train_step_not_entail = df_train_step_not_entail.sample(n=min(len(df_train_step), len(df_train_step_not_entail)), random_state=random_seed)
    df_train_step_not_entail["hypothesis"] = [hypothesis] * len(df_train_step_not_entail)
    df_train_step_not_entail["label"] = [1] * len(df_train_step_not_entail)
    # append
    df_train_lst.append(pd.concat([df_train_step, df_train_step_not_entail]))
  df_train = pd.concat(df_train_lst)

  # shuffle
  df_train = df_train.sample(frac=1, random_state=random_seed)
  df_train["label"] = df_train.label.apply(int)
  df_train["label_nli_explicit"] = ["True" if label == 0 else "Not-True" for label in df_train["label"]]  # adding this just to simplify readibility

  print(f"After adding not_entailment training examples, the training data was augmented to {len(df_train)} texts.")
  print(f"Max augmentation could be: len(df_train) * 2 = {length_original_data_train*2}. It can also be lower, if there are more entail examples than not-entail for a majority class.")

  return df_train.copy(deep=True)


df_train_formatted = format_nli_trainset(df_train=df_train, hypo_label_dic=hypothesis_label_dic, random_seed=SEED_GLOBAL)

Length of df_train before formatting step: 1600.
After adding not_entailment training examples, the training data was augmented to 2220 texts.
Max augmentation could be: len(df_train) * 2 = 3200. It can also be lower, if there are more entail examples than not-entail for a majority class.


**Inspect reformatted training dataset**

Label 0 means that the hypothesis is 'true', label 1 means that the hypothesis is 'not-true'.


In [ ]:
DataTable(df_train_formatted[["label", "label_nli_explicit", "hypothesis", "text_prepared"]], num_rows_per_page=5)

**Format the test set**

To know which class-hypothesis is true for a specific text, we need to test every possible class-hypothesis for each text. We therefore multiple the rows/texts in the test set by the number of hypothesis and pair each text with all possible hypotheses. The table below shows what the reformatted test set looks like.

In [ ]:
## function for reformatting the test set
def format_nli_testset(df_test=None, hypo_label_dic=None):
  ## explode test dataset for N hypotheses
  hypothesis_lst = [value for key, value in hypo_label_dic.items()]
  print("Number of hypotheses/classes: ", len(hypothesis_lst))

  # label lists with 0 at alphabetical position of their true hypo, 1 for not-true hypos
  label_text_label_dic_explode = {}
  for key, value in hypo_label_dic.items():
    label_lst = [0 if value == hypo else 1 for hypo in hypothesis_lst]
    label_text_label_dic_explode[key] = label_lst

  df_test["label"] = df_test.label_text.map(label_text_label_dic_explode)
  df_test["hypothesis"] = [hypothesis_lst] * len(df_test)
  print(f"Original test set size: {len(df_test)}")

  # explode dataset to have K-1 additional rows with not_entail label and K-1 other hypotheses
  # ! after exploding, cannot sample anymore, because distorts the order to true label values, which needs to be preserved for evaluation code
  df_test = df_test.explode(["hypothesis", "label"])  # multi-column explode requires pd.__version__ >= '1.3.0'
  print(f"Test set size for NLI classification: {len(df_test)}\n")

  df_test["label_nli_explicit"] = ["True" if label == 0 else "Not-True" for label in df_test["label"]]  # adding this just to simplify readibility

  return df_test.copy(deep=True)


df_test_formatted = format_nli_testset(df_test=df_test, hypo_label_dic=hypothesis_label_dic)

Number of hypotheses/classes:  2
Original test set size: 400
Test set size for NLI classification: 800



**Inspect the reformatted test dataset**


In [ ]:
DataTable(df_test_formatted[["label", "label_nli_explicit", "hypothesis", "text_prepared"]].sort_values(["text_prepared", "hypothesis"]), num_rows_per_page=6, max_rows=10_000)

## Fine-tuning

We use [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) for loading and training our model. They provide great documentation and also a very good [course](https://huggingface.co/course/chapter1/1) on how to use Transformers.

**Loading an NLI model**

You can can use any NLI model on the Hugging Face Hub. For normal English use-cases, we recommend this [base-size model](https://huggingface.co/MoritzLaurer/DeBERTa-v3-base-mnli-fever-docnli-ling-2c); for multilingual/non-English use-cases, we recommend this [multilingual model](https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7); for best performance in English (but high compute and memory requirements) we recommend this [large model](https://huggingface.co/MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli).


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

## load the BERT-NLI model and its tokenizer
# you can choose any of the NLI models here: https://huggingface.co/MoritzLaurer
model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, model_max_length=512)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# use GPU (cuda) if available, otherwise use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
model.to(device);


Device: cuda


**Tokenize data**

In [ ]:
# convert pandas dataframes to Hugging Face dataset object to facilitate pre-processing
import datasets

dataset = datasets.DatasetDict({
    "train": datasets.Dataset.from_pandas(df_train_formatted),
    "test": datasets.Dataset.from_pandas(df_test_formatted)
})

# tokenize
def tokenize_nli_format(examples):
  return tokenizer(examples["text_prepared"], examples["hypothesis"], truncation=True, max_length=512)  #512 max_length can be reduced to e.g. 256 to increase speed, but long texts will be cut off
dataset["train"] = dataset["train"].map(tokenize_nli_format, batched=True)
dataset["test"] = dataset["test"].map(tokenize_nli_format, batched=True)

# remove unnecessary columns for model training

dataset = dataset.remove_columns(['label_text','Unnamed: 0', 'X', 'date', 'text', 'user_id','Zperiod', 'lang', 'source','descUAid', 'descRUid', 'solUAid', 'hostRUid', 'hostUAid',
       'solRUid', 'Unnamed: 14', '__index_level_0__'])


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

**Inspect processed data**

In [ ]:
print("The overall structure of the pre-processed train and test sets:\n")
print(dataset)

print("\n\nAn example for a tokenized hypothesis-context pair:\n")
print(dataset["train"][0])

The overall structure of the pre-processed train and test sets:

DatasetDict({
    train: Dataset({
        features: ['label', 'text_prepared', 'hypothesis', 'label_nli_explicit', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2220
    })
    test: Dataset({
        features: ['label', 'text_prepared', 'hypothesis', 'label_nli_explicit', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 800
    })
})


An example for a tokenized hypothesis-context pair:

{'label': 1, 'text_prepared': 'The quote: "У ОК "Південь" розповіли про наявність можливостей щодо знищення окупантів<url>" - end of the quote.', 'hypothesis': 'The quote is not about hostility towards Russia or Russians', 'label_nli_explicit': 'Not-True', 'input_ids': [1, 487, 66640, 268, 314, 3901, 260, 44769, 314, 3252, 1078, 68136, 312, 51307, 123058, 924, 260, 1966, 98453, 260, 44847, 39792, 1497, 4023, 260, 40318, 36476, 59340, 11967, 1078, 2710, 1475, 670, 312, 260, 265, 3163, 305, 288, 66

### Setting training arguments / hyperparameters

The following cell sets several important hyperparameters. We chose parameters that work well in general to avoid the need for hyperparameter search. Further below, we also provide code for hyperparameter search, if researchers want to try to increase performance by a few percentage points.

In [ ]:
from transformers import TrainingArguments, Trainer, logging

# Set the directory to write the fine-tuned model and training logs to.
# With google colab, this will create a temporary folder, which will be deleted once you disconnect.
# You can connect to your personal google drive to save models and logs properly.
training_directory = "BERT-nli-ua-host"

# FP16 is a hyperparameter which can increase training speed and reduce memory consumption, but only on GPU and if batch-size > 8, see here: https://huggingface.co/transformers/performance.html?#fp16
# FP16 does not work on CPU or for multilingual mDeBERTa models
fp16_bool = True if torch.cuda.is_available() else False
if "mdeberta" in model_name.lower(): fp16_bool = False  # multilingual mDeBERTa does not support FP16 yet: https://github.com/microsoft/DeBERTa/issues/77
# in case of hyperparameter search end the end: FP16 has to be set to False. The integrated hyperparameter search with the Hugging Face Trainer can lead to errors otherwise.
fp16_bool = False

# Hugging Face tipps to increase training speed and decrease out-of-memory (OOM) issues: https://huggingface.co/transformers/performance.html?
# Overview of all training arguments: https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments
train_args = TrainingArguments(
    output_dir=f'./results/{training_directory}',
    logging_dir=f'./logs/{training_directory}',
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # if you get an out-of-memory error, reduce this value to 8 or 4 and restart the runtime. Higher values increase training speed, but also increase memory requirements. Ideal values here are always a multiple of 8.
    per_device_eval_batch_size=32,  # if you get an out-of-memory error, reduce this value, e.g. to 40 and restart the runtime
    #gradient_accumulation_steps=4, # Can be used in case of memory problems to reduce effective batch size. accumulates gradients over X steps, only then backward/update. decreases memory usage, but also slightly speed. (!adapt/halve batch size accordingly)
    num_train_epochs=10,  # this can be increased, but higher values increase training time. Good values for NLI are between 3 and 20.
    warmup_ratio=0.25,  # a good normal default value is 0.06 for normal BERT-base models, but since we want to reuse prior NLI knowledge and avoid catastrophic forgetting, we set the value higher
    weight_decay=0.1,
    seed=SEED_GLOBAL,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=fp16_bool,  # Can speed up training and reduce memory consumption, but only makes sense at batch-size > 8. loads two copies of model weights, which creates overhead. https://huggingface.co/transformers/performance.html?#fp16
    fp16_full_eval=fp16_bool,
    evaluation_strategy="no", # options: "no"/"steps"/"epoch"
    #eval_steps=10_000,  # evaluate after n steps if evaluation_strategy!='steps'. defaults to logging_steps
    save_strategy = "no",  # options: "no"/"steps"/"epoch"
    #save_steps=10_000,              # Number of updates steps before two checkpoint saves.
    #save_total_limit=10,             # If a value is passed, will limit the total amount of checkpoints. Deletes the older checkpoints in output_dir
    #logging_strategy="steps",
    report_to="all",  # "all"  # logging
    #push_to_hub=False,
    #push_to_hub_model_id=f"{model_name}-finetuned-{task}",
)


In [ ]:
# helper function to clean memory and reduce risk of out-of-memory error
import gc
def clean_memory():
  #del(model)
  if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
  gc.collect()

clean_memory()

### Custom function to compute metrics for NLI

We multiplied each text N times for each class in the test set and NLI can only predict 2 or 3 classes: true/not-true or true/neutral/false. This means that we cannot use standard functions for computing metrics. The following function reformats the model's output in a way that allows for the calculation of standard metrics like accuracy, F1-macro etc.

In [ ]:
from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, accuracy_score, classification_report

def compute_metrics_nli_binary(eval_pred, label_text_alphabetical=None):
    predictions, labels = eval_pred

    ### reformat model output to enable calculation of standard metrics
    # split in chunks with predictions for each hypothesis for one unique premise
    def chunks(lst, n):  # Yield successive n-sized chunks from lst. https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # for each chunk/premise, select the most likely hypothesis
    softmax = torch.nn.Softmax(dim=1)
    prediction_chunks_lst = list(chunks(predictions, len(set(label_text_alphabetical)) ))
    hypo_position_highest_prob = []
    for i, chunk in enumerate(prediction_chunks_lst):
        hypo_position_highest_prob.append(np.argmax(np.array(chunk)[:, 0]))  # only accesses the first column of the array, i.e. the entailment/true prediction logit of all hypos and takes the highest one

    label_chunks_lst = list(chunks(labels, len(set(label_text_alphabetical)) ))
    label_position_gold = []
    for chunk in label_chunks_lst:
        label_position_gold.append(np.argmin(chunk))  # argmin to detect the position of the 0 among the 1s

    print("Highest probability prediction per premise: ", hypo_position_highest_prob)
    print("Correct label per premise: ", label_position_gold)

    ### calculate standard metrics
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(label_position_gold, hypo_position_highest_prob, average='macro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(label_position_gold, hypo_position_highest_prob, average='micro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
    acc_balanced = balanced_accuracy_score(label_position_gold, hypo_position_highest_prob)
    acc_not_balanced = accuracy_score(label_position_gold, hypo_position_highest_prob)
    metrics = {'f1_macro': f1_macro,
               'f1_micro': f1_micro,
               'accuracy_balanced': acc_balanced,
               'accuracy_not_b': acc_not_balanced,
               #'precision_macro': precision_macro,
               #'recall_macro': recall_macro,
               #'precision_micro': precision_micro,
               #'recall_micro': recall_micro,
               #'label_gold_raw': label_position_gold,
               #'label_predicted_raw': hypo_position_highest_prob
               }
    print("Aggregate metrics: ", {key: metrics[key] for key in metrics if key not in ["label_gold_raw", "label_predicted_raw"]} )  # print metrics but without label lists
    print("Detailed metrics: ", classification_report(label_position_gold, hypo_position_highest_prob, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]), target_names=label_text_alphabetical, sample_weight=None, digits=2, output_dict=True,
                                zero_division='warn'), "\n")
    return metrics

# Create alphabetically ordered list of the original dataset classes/labels
# This is necessary to be sure that the ordering of the test set labels and predictions is the same. Otherwise there is a risk that labels and predictions are in a different order and resulting metrics are wrong.
label_text_alphabetical = np.sort(df_train.label_text.unique())


### Fine-tuning and evaluation  (IGNORE THIS AND PROCEED TO HP SEARCH)

Let's start fine-tuning the model!

If you get an 'out-of-memory' error, reduce the 'per_device_train_batch_size' to 8 or 4 in the TrainingArguments above and restart the runtime. If you don't restart your runtime (menu to the to left 'Runtime' > 'Restart runtime') and rerun the entire script, the 'out-of-memory' error will probably not go away.

In [ ]:
# training
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=train_args,
    train_dataset=dataset["train"],  #.shard(index=1, num_shards=100),  # could shard data for faster testing https://huggingface.co/docs/datasets/processing.html#sharding-the-dataset-shard
    eval_dataset=dataset["test"],  #.shard(index=1, num_shards=100),
    compute_metrics=lambda eval_pred: compute_metrics_nli_binary(eval_pred, label_text_alphabetical=label_text_alphabetical)
)

trainer.train()


In [ ]:
## Evaluate the fine-tuned model on the held-out test set
results = trainer.evaluate()

In [ ]:
print(results)

## Hyperparameter Search

To increase performance, you can also conduct a hyperparameter search (hp-search), to try and find the best hyperparameters for your specific task and dataset. The trade-off is that hp-search is very compute intensive, but finding better hyperparameters for your task can increase performance. Make sure to conduct hp-search on a sub-set of the training set (i.e. validation set) and not the final test set to avoid data leakage of the test set before final testing.

Note that for small datasets, running the hp-search only on one train-validation split is not ideal. For datasets with less than around 2000 training data points, we recommend running the hp-search on two different random train-validation split. We implemented this for our paper, but not in this notebook as this would make the code harder to understand.

Documentation with more information on hp-search with Hugging Face Transformers is available [here](https://huggingface.co/docs/transformers/main/hpo_train).

In [ ]:
## train-validation split - test set should not be visible during hp-search
# https://huggingface.co/docs/datasets/v2.5.1/en/package_reference/main_classes#datasets.Dataset.train_test_split

# the ideal size of the validation set depends on the size of your training data. Each label should have at the very least a few dozen examples in the validation set (ideally several hundred)
validation_set_size = 0.4  # for a training data size of 1000 with 3 classes we use 40% of the training data for validating hyperparameters

# reformatting of label column to enable dataset stratification
from datasets import ClassLabel
new_features = dataset["train"].features.copy()
if len(model.config.label2id.keys()) == 2:  # for 2-class NLI model
  label_names = ["entailment", "not-entailment"]
elif len(model.config.label2id.keys()) == 3:  # for 3-class NLI model
  label_names = ["entailment", "neutral", "contradiction"]
new_features['label'] = ClassLabel(names=label_names)
print(len(dataset['train']))
dataset = dataset.cast(new_features)
print(len(dataset['train']))

# train-validation split for hp-search
dataset_hp = dataset["train"].train_test_split(test_size=validation_set_size, seed=SEED_GLOBAL, shuffle=True, stratify_by_column="label")
print(dataset_hp)

2220


Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

1000
DatasetDict({
    train: Dataset({
        features: ['label', 'text_prepared', 'hypothesis', 'label_nli_explicit', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 600
    })
    test: Dataset({
        features: ['label', 'text_prepared', 'hypothesis', 'label_nli_explicit', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400
    })
})


In [ ]:
## Reinitialize trainer for hp-search
# https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10

def model_init():
  clean_memory()
  return AutoModelForSequenceClassification.from_pretrained(model_name).to(device)  # return_dict=True

trainer = Trainer(
    model_init=model_init,
    tokenizer=tokenizer,
    args=train_args,
    train_dataset=dataset_hp["train"],
    eval_dataset=dataset_hp["test"],
    compute_metrics=lambda eval_pred: compute_metrics_nli_binary(eval_pred, label_text_alphabetical=label_text_alphabetical)
);


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--MoritzLaurer--mDeBERTa-v3-base-xnli-multilingual-nli-2mil7/snapshots/12dfea2a0b9ea8019b809105d4f62ffec7af622c/config.json
Model config DebertaV2Config {
  "_name_or_path": "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7",
  "architectures": [
    "DebertaV2ForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "neutral",
    "2": "contradiction"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_

**Define the hyperparameters you want to optimise**

For a detailed discussion of different hyperparameters, see the appendix of our paper.

In [ ]:
# we use Optuna for hp-search: https://optuna.readthedocs.io/en/stable/
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [9e-6, 2e-5, 4e-5]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 4, 16, log=False, step=2),   # increasing the maximum number of epochs here could increase performance but will take (much) longer to train
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.1, 0.6, log=True),
        "per_device_train_batch_size": 8,  # lower this value in case of out-of-memory errors and restart the runtime
        #"per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
    }


**Run HP search!**

Choose the number of hyperparameter configurations you want to test. In our experiments we found that after 10 to 15 trials with around 4 hyperparameters, performance is unlikely to increase meaningfully. 15 trials seems to be a safe value, but can take a while to run.

In [ ]:
import optuna

# number of differen hp configurations to test
numer_of_trials = 16  # increasing this value can lead to better hyperparameters, but will take longer
# chose the sampler for sampling hp configurations
optuna_sampler = optuna.samplers.TPESampler(seed=SEED_GLOBAL, consider_prior=True, prior_weight=1.0, consider_magic_clip=True, consider_endpoints=False, n_startup_trials=numer_of_trials/2, n_ei_candidates=24, multivariate=False, group=False, warn_independent_sampling=True, constant_liar=False)  # https://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.TPESampler.html#optuna.samplers.TPESampler

# Hugging Face Documentation: https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.hyperparameter_search
best_run = trainer.hyperparameter_search(
    n_trials=numer_of_trials,
    direction="maximize",
    hp_space=my_hp_space,
    backend='optuna',
    **{"sampler": optuna_sampler}
)

[I 2023-05-22 16:17:48,626] A new study created in memory with name: no-name-2211e31d-34ba-4a89-aead-be33cc6d498d
Trial: {'learning_rate': 2e-05, 'num_train_epochs': 12, 'warmup_ratio': 0.13225317293225414}
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--MoritzLaurer--mDeBERTa-v3-base-xnli-multilingual-nli-2mil7/snapshots/12dfea2a0b9ea8019b809105d4f62ffec7af622c/config.json
Model config DebertaV2Config {
  "_name_or_path": "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7",
  "architectures": [
    "DebertaV2ForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "neutral",
    "2": "contradiction"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 51

Step,Training Loss
500,0.267200




Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: hypothesis, label_nli_explicit, text_prepared. If hypothesis, label_nli_explicit, text_prepared are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32


[I 2023-05-22 16:23:58,272] Trial 0 finished with value: 2.796157877706835 and parameters: {'learning_rate': 2e-05, 'num_train_epochs': 12, 'warmup_ratio': 0.13225317293225414}. Best is trial 0 with value: 2.796157877706835.
Trial: {'learning_rate': 4e-05, 'num_train_epochs': 12, 'warmup_ratio': 0.3556211334079358}


Highest probability prediction per premise:  [1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]
Correct label per premise:  [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0

...

Highest probability prediction per premise:  [1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
Correct label per premise:  [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--MoritzLaurer--mDeBERTa-v3-base-xnli-multilingual-nli-2mil7/snapshots/12dfea2a0b9ea8019b809105d4f62ffec7af622c/config.json
Model config DebertaV2Config {
  "_name_or_path": "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7",
  "architectures": [
    "DebertaV2ForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "neutral",
    "2": "contradiction"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_

Step,Training Loss
500,0.273800




Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: hypothesis, label_nli_explicit, text_prepared. If hypothesis, label_nli_explicit, text_prepared are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
[I 2023-05-22 17:45:20,769] Trial 15 finished with value: 2.9458910270589103 and parameters: {'learning_rate': 2e-05, 'num_train_epochs': 8, 'warmup_ratio': 0.16352751048458855}. Best is trial 15 with value: 2.9458910270589103.


Highest probability prediction per premise:  [1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]
Correct label per premise:  [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0

In [ ]:
# show best hyperparameters based on hp-search
print(best_run)

BestRun(run_id='15', objective=2.9458910270589103, hyperparameters={'learning_rate': 2e-05, 'num_train_epochs': 8, 'warmup_ratio': 0.16352751048458855})


**Training Time with optimised hyperparameters!**

Here we can use the original train and test set again.

In [ ]:
# update the training arguments with the best hyperparameters
for k,v in best_run.hyperparameters.items():
    setattr(train_args, k, v)
print("\n", train_args)

# hp-search with hf causes errors with FP16 for some reason
#setattr(train_args, "fp16", False)
#setattr(train_args, "fp16_full_eval", False)


 TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_n

In [ ]:
dataset = datasets.DatasetDict({
    "train": datasets.Dataset.from_pandas(df_train_formatted),
    "test": datasets.Dataset.from_pandas(df_test_formatted)
})

# tokenize
def tokenize_nli_format(examples):
  return tokenizer(examples["text_prepared"], examples["hypothesis"], truncation=True, max_length=512)  #512 max_length can be reduced to e.g. 256 to increase speed, but long texts will be cut off
dataset["train"] = dataset["train"].map(tokenize_nli_format, batched=True)
dataset["test"] = dataset["test"].map(tokenize_nli_format, batched=True)

# remove unnecessary columns for model training

dataset = dataset.remove_columns(['label_text','Unnamed: 0', 'X', 'date', 'text', 'user_id','Zperiod', 'lang', 'source','descUAid', 'descRUid', 'solUAid', 'hostRUid', 'hostUAid',
       'solRUid', 'Unnamed: 14', '__index_level_0__'])

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# Training
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=train_args,
    train_dataset=dataset["train"],  #.shard(index=1, num_shards=100),  # https://huggingface.co/docs/datasets/processing.html#sharding-the-dataset-shard
    eval_dataset=dataset["test"],  #.shard(index=1, num_shards=100),
    compute_metrics=lambda eval_pred: compute_metrics_nli_binary(eval_pred, label_text_alphabetical=label_text_alphabetical)
)

trainer.train()


The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: hypothesis, label_nli_explicit, text_prepared. If hypothesis, label_nli_explicit, text_prepared are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2220
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2224
/usr/local/lib/python3.10/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:

Step,Training Loss
500,0.470100
1000,0.195300
1500,0.065100
2000,0.022300




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2224, training_loss=0.17064119007090012, metrics={'train_runtime': 944.8531, 'train_samples_per_second': 18.797, 'train_steps_per_second': 2.354, 'total_flos': 1611388912695120.0, 'train_loss': 0.17064119007090012, 'epoch': 8.0})

In [ ]:
## Evaluate the fine-tuned model on the held-out test set
results = trainer.evaluate()


In [ ]:
print(results)

{'eval_loss': 1.0843173265457153, 'eval_f1_macro': 0.8052970651395848, 'eval_f1_micro': 0.8725, 'eval_accuracy_balanced': 0.8188007644529384, 'eval_accuracy_not_b': 0.8725, 'eval_runtime': 13.78, 'eval_samples_per_second': 58.055, 'eval_steps_per_second': 1.814, 'epoch': 8.0}


## Save and load your fine-tuned model

This segment provides code for saving the model to your hard-disk or for uploading it to the Hugging Face hub.

In [ ]:
## first you need to connect to your google drive with your google account
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=False)
#drive.flush_and_unmount()

# insert the path where you want to save the model
os.chdir("/content/drive/My Drive/")
print(os.getcwd())


In [ ]:
model_custom_path = "content/drive/My Drive/YOUR PATH HERE"
trainer.save_model(output_dir=model_custom_path)

In [ ]:
### Push to hub
# install necessary dependencies
!sudo apt-get install git-lfs
!huggingface-cli login

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid.
Cannot authenticate through git-credential as no

In [ ]:
# load your models from disk
model = AutoModelForSequenceClassification.from_pretrained(model_custom_path)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, model_max_length=512)  # we load the tokenizer from the original BERT-NLI model

# https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.push_to_hub
repo_id = 'enter your repo id'
model.push_to_hub(repo_id=repo_id)
tokenizer.push_to_hub(repo_id=repo_id)
